## Import needed modules

In [83]:
import numpy as np
import pandas as pd

np.random.seed(1)    

## Initialize data

In [84]:
data = pd.read_csv('../data/heart_statlog_cleveland_hungary_final.csv')
X = np.array(data.drop(columns = ['target']))
y = np.array(pd.DataFrame(data['target']))

## Define activation function

In [85]:
# Sigmoid Activation Function
# If deriv = True, x value must be output of a sigmoid function to be accurate
def Sigmoid(x, deriv = False):
    if(deriv):
        return x * (1-x)
    return 1/(1+np.exp(-x))

## Define Neural Network Function

In [100]:
# Binary Classification Neural Network
# X: Numpy array, each row is one data point, each column is one input
# y: Numpy array, each row is one response variable, one column, values are either 0 or 1
# hiddenlayers: array containing size of hiddenlayers
# learning_rate: number scaling (usually down) the speed of learning
# non_linear: function for activation function with deriv parameter to return derivative given the output
# Returns: tuple of weights, estimated probabilities, and final mean fitted error
def NeuralNetwork(X, y, hiddenlayers, learning_rate = 0.01, iterations = 60000, non_linear = Sigmoid):
    X = (X-X.min(0)) / X.ptp(0) # Normalize data

    # Dimensions of data
    n, inputs = X.shape
    
    sizes = hiddenlayers
    sizes.insert(0, inputs)
    sizes.append(1)
    
    # Initialize weights
    weights_list = []
    for j in range(len(sizes) - 1):
        weights = 2 * np.random.rand(sizes[j], sizes[j+1]) - 1 # Weights from layer j to layer j+1
        weights_list.append(weights)
    
    for j in range(iterations):
        layers = [X]
        for i in range(len(weights_list)):
            new_layer = non_linear(np.dot(layers[-1], weights_list[i]))
            layers.append(new_layer)
    
        # Backpropagation
        global_error = y - layers[-1]
        global_delta = global_error * Sigmoid(layers[-1], deriv = True)
        
        if (j % round((iterations / 6))) == 0:
            print("Error: " + str(np.mean(np.abs(global_error))))

        # Calculate error contributions starting from last layers
        layer_deltas = [global_delta]
        for i in range(1, len(weights_list)+1):
            layer_error = np.dot(layer_deltas[0], weights_list[-i].T)
            layer_delta = layer_error * Sigmoid(layers[-i-1], deriv = True)
    
            layer_deltas.insert(0, layer_delta)

        # Update weights
        for i in range(1, len(weights_list)+1):
            weights_list[-i] += learning_rate * np.dot(layers[-i-1].T, layer_deltas[-i])

    return (weights_list, layers[-1], np.mean(np.abs(global_error)))

In [101]:
weights_list, y_probs, mean_error = NeuralNetwork(X, y, [8,8])

Error: 0.5214351204212726
Error: 0.12453082771649909
Error: 0.10810920545822855
Error: 0.0926540326152382
Error: 0.08856723627674871
Error: 0.07038599753787227


In [102]:
print(mean_error)

0.060735202552720316
